# How to learn pressure term

In [1]:
### Learn pressure term through its gradient:
###     Setup neural network, train through gradient
###     - Function that evaluates gradient
###     - Training data: Average of (Unew + Uforcing term)/dt
###     - To lock down the constant: Need condition
###         - Sum of pressure = 0
###         - Pressure at a point = 0

In [2]:
import time
import numpy as np
import math

import torch
import torch.nn as nn
import torch.optim as optim

import DRLPDE_nn
import DRLPDE_functions.DefineDomain
import DRLPDE_functions.EvaluateWalkers

In [3]:
# Time step
dt = 1e-4

# exit tolerance
tol = 1e-6

# Number of walkers
num_walkers = 2**13
num_ghost = 256
num_batch = 2**11

# Update walkers
# Options: 
#    move -- moves walkers to one of their new locations
#    remake -- remake walkers at each training step
#    fixed -- keeps walkers fixed
update_walkers = 'move'
update_walkers_every = 1

############## Training Parameters #######################

# Training epochs
num_epoch = 1000
update_print_every = 1000

# Neural Network Architecture
nn_depth = 20
nn_width = 4

# Weighting of losses
lambda_bell = 1e-2/dt
lambda_fix = 1

# Learning rate
learning_rate = 1e-2
adam_beta = (0.9,0.999)
weight_decay = 0

In [4]:
import importlib
    
DRLPDE_param = importlib.import_module(".JCPexample6", package='examples')
    
### Use cuda
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

boundingbox = DRLPDE_param.boundingbox
list_of_dirichlet_boundaries = DRLPDE_param.list_of_dirichlet_boundaries
list_of_periodic_boundaries = DRLPDE_param.list_of_periodic_boundaries
pde_type = DRLPDE_param.pde_type
is_unsteady = DRLPDE_param.is_unsteady
output_dim = DRLPDE_param.output_dim

nn_param = {'depth': nn_depth,
            'width': nn_width,
            'x_dim':DRLPDE_param.x_dim,
            'is_unsteady':DRLPDE_param.is_unsteady ,
            'output_dim': 1
            }
            
x_dim = DRLPDE_param.x_dim
mu = DRLPDE_param.mu

eval_model_param={'dt': dt,
                  'forcing': DRLPDE_param.forcing}

### Import functions

################ Preparing the model #################

#print("Initializing the model")

### Make boundaries defining the domain
Domain = DRLPDE_functions.DefineDomain.Domain(is_unsteady, boundingbox, 
                                                list_of_dirichlet_boundaries,
                                                list_of_periodic_boundaries)

model_velocity = torch.load("savedmodels/JCPexample6.pt")

### Initialize the Model
MyNeuralNetwork = DRLPDE_nn.FeedForwardNN
model_pressure = MyNeuralNetwork(**nn_param).to(dev)

mseloss = nn.MSELoss(reduction = 'mean')
optimizer = optim.Adam(model_pressure.parameters(), 
                        lr=learning_rate, 
                        betas=adam_beta, 
                        weight_decay=weight_decay)

### Create Walkers and Boundary points and Organize into DataLoader
RWalkers = DRLPDE_functions.DefineDomain.Walker_Data(num_walkers, boundingbox, Domain.boundaries)
RWalkers_batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)

if update_walkers == 'move':
    move_RWalkers = torch.zeros_like(RWalkers.location)

###
fix_pressure_at = RWalkers.location[0].to(dev)

In [5]:
### Functions
def eval_gradient(X, model):
    
    a = model(X)

    grad_model = torch.autograd.grad(a, X, grad_outputs = torch.ones_like(a), create_graph = True, 
                                       retain_graph = True, only_inputs = True)[0]

    return grad_model

def move_Walkers(X, model, Domain, x_dim, mu, dt, num_batch, num_ghost):
    
    ### Evaluate model
    Uold = model(X)

    ### Move walkers
    Zt = np.sqrt(dt)*torch.randn((num_batch*num_ghost, x_dim), device=X.device, requires_grad=True)
    
    Xnew = X.repeat(num_ghost,1) - dt*Uold.detach().repeat(num_ghost,1) + np.sqrt(2*mu)*Zt

    ### Calculate exits
    outside = torch.zeros( Xnew.size(0), dtype=torch.bool, device=Xnew.device)

    for bdry in Domain.boundaries:
        outside_bdry = bdry.dist_to_bdry(Xnew) < 0
        outside += outside_bdry

    return Xnew, outside[:num_batch]

In [6]:
start_time = time.time()

for step in range(num_epoch):

    # Random Walkers - do in batches
    for Xold, index in RWalkers_batch:

        # Send to GPU and set requires grad flag
        Xold = Xold.to(dev).requires_grad_(True)

        # Evaluate at old location and Move walkers
        Xnew, outside = move_Walkers(Xold, model_velocity, Domain, x_dim, mu, dt, num_batch, num_ghost)

        # Evaluate at new location and average
        Unew = model_velocity(Xnew).reshape(num_ghost, num_batch, output_dim).mean(0)

        grad_pressure = eval_gradient(Xold, model_pressure)

        # Calculate loss
        loss = lambda_bell*mseloss(grad_pressure, Unew.detach()/dt)
        loss.backward()

        # If moving walkers save the first ghost walker
        if update_walkers == 'move':
            if any(outside):
                Xnew[:num_batch,:][outside,:] = DRLPDE_functions.DefineDomain.generate_interior_points(torch.sum(outside), 
                                                                                            boundingbox,
                                                                                            Domain.boundaries).to(dev)
            move_RWalkers[index,:] = Xnew[:num_batch].detach().cpu()

    # Try fixing one point to have 0 pressure
    loss = lambda_fix*mseloss( model_pressure(fix_pressure_at), torch.zeros(1, device=dev))
    loss.backward()

    # Make optimization step
    optimizer.step()
    optimizer.zero_grad()

    # Update walkers

    if (step+1) % update_walkers_every == 0:
        if update_walkers == 'move':
            RWalkers.location = move_RWalkers
            RWalkers_Batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)
        elif update_walkers == 'remake':
            RWalkers = DRLPDE_functions.DefineDomain.Walker_Data(num_walkers, boundingbox, Domain.boundaries)
            RWalkers_Batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)

    if step == 0:
        print('No errors in first epoch: Training will continue')
    if (step+1) % update_print_every == 0:
        current_time = time.time()
        np.set_printoptions(precision=2)
        print('step = {0}/{1}, {2:2.3f} s/step, time-to-go:{3:2.0f}s'.format(
                step+1, num_epoch, (current_time - start_time) / (step + 1), 
            (current_time - start_time) / (step + 1) * (num_epoch - step - 1)))
    

No errors in first epoch: Training will continue
step = 1000/1000, 0.197 s/step, time-to-go: 0s


In [7]:
torch.save(model_pressure, "savedmodels/" + 'pressure_test'+ ".pt")